BI Goals:

i) investigar as ligações entre a atividade do fogo, conforme medido pelo FRP (Fire Radiative Power), e as concentrações de poluentes e avaliar a zona espacial e temporal de influência da atividade dos incêndios florestais.

(ii) investigar a utilização de FRP como ferramenta para filtrar a contribuição do fumo de biomassa para os registos de poluição atmosférica em bacias atmosféricas urbanas, nomeadamente as emissões de carbono resultantes de incêndios florestais graves.

(iii) desenvolver abordagens multirriscos para caracterizar o comportamento conjunto de múltiplos perigos e riscos consequentes e avaliar o papel desempenhado por condições anteriores e simultâneas de seca e/ou calor na exacerbação de incêndios rurais e consequentes ondas de fumo

Combine datasets (labeled dataset that contains info about labels and FRP & pollutants statistics) by day and pixel

In [1]:
import xarray as xr
import pandas as pd
import numpy as np

# ==== File paths ====
pm25_path = r"D:\IPMA\CAMS\chem_singlvl\daily_pm2p5_stats.nc"
fire_path = r"D:\IPMA\FRP\fire_labels_by_region\fire_data_Spain.nc"
mask_path = r"D:\IPMA\Countries\Spain_mask.nc" 
output_path = r"D:\IPMA\CAMS\pm2p5_fire_Spain.nc"

# ==== Load datasets ====
ds_pm25 = xr.open_dataset(pm25_path)
ds_fire = xr.open_dataset(fire_path)
mask_ds = xr.open_dataset(mask_path)

# ==== Load mask variable ====
# Assuming mask variable is 1 inside the country and 0 outside
mask = mask_ds["mask"]  # change "mask" if your variable name is different

# ==== Extract Year, Month, Day ====
years = ds_pm25['Year'].values
months = ds_pm25['Month'].values
days = ds_pm25['Day'].values

# ==== Create all combinations ====
ymd = pd.MultiIndex.from_product(
    [years, months, days],
    names=['year', 'month', 'day']
).to_frame(index=False)

# ==== Remove invalid dates ====
def is_valid_date(row):
    try:
        pd.Timestamp(year=int(row['year']), month=int(row['month']), day=int(row['day']))
        return True
    except ValueError:
        return False

valid_mask = ymd.apply(is_valid_date, axis=1)
ymd_valid = ymd[valid_mask].reset_index(drop=True)

# ==== Convert valid dates to datetime ====
time_index = pd.to_datetime(ymd_valid)

# ==== Flatten pollutant data ====
ds_pm25_stack = ds_pm25.stack(time=('Year', 'Month', 'Day'))

# Keep only valid dates
ds_pm25_stack = ds_pm25_stack.isel(time=valid_mask.values)

# Assign new datetime coordinate
ds_pm25_stack = ds_pm25_stack.assign_coords(time=time_index)

# ==== Select overlapping time ====
start_time = max(ds_pm25_stack.time.values[0], ds_fire.time.values[0])
end_time = min(ds_pm25_stack.time.values[-1], ds_fire.time.values[-1])

ds_pm25_sel = ds_pm25_stack.sel(time=slice(start_time, end_time))
ds_fire_sel = ds_fire.sel(time=slice(start_time, end_time))

# ==== Check lat/lon match ====
assert np.allclose(ds_pm25_sel.latitude.values, ds_fire_sel.latitude.values), "Latitude mismatch"
assert np.allclose(ds_pm25_sel.longitude.values, ds_fire_sel.longitude.values), "Longitude mismatch"
assert np.allclose(ds_pm25_sel.latitude.values, mask.latitude.values), "Latitude mismatch with mask"
assert np.allclose(ds_pm25_sel.longitude.values, mask.longitude.values), "Longitude mismatch with mask"

# ==== Apply mask to both datasets ====
ds_pm25_masked = ds_pm25_sel.where(mask == 1)
ds_fire_masked = ds_fire_sel.where(mask == 1)

# ==== Merge datasets ====
ds_merged = xr.merge([ds_pm25_masked, ds_fire_masked])

# ==== Save ====
ds_merged.to_netcdf(output_path)
print(f"Merged dataset saved to {output_path}")


C:\Users\sofia\AppData\Local\Temp\ipykernel_4940\3325753227.py:52: FutureWarning: updating coordinate 'time' with a PandasMultiIndex would leave the multi-index level coordinates ['Year', 'Month', 'Day'] in an inconsistent state. This will raise an error in the future. Use `.drop_vars(['time', 'Year', 'Month', 'Day'])` before assigning new coordinate values.
  ds_pm25_stack = ds_pm25_stack.assign_coords(time=time_index)


Merged dataset saved to D:\IPMA\CAMS\pm2p5_fire_Spain.nc
